# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
!pip3 install numpy

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
import numpy as np
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [3]:
stocks = pd.read_csv('./sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [4]:
from secretsofalgotr import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [5]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
# print(data.status_code == 200)

In [6]:
data

{'avgTotalVolume': 82309831,
 'calculationPrice': 'close',
 'change': -3.08,
 'changePercent': -0.02211,
 'close': 147.22,
 'closeSource': 'flicaofi',
 'closeTime': 1731897502912,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': 143.67,
 'delayedPriceTime': 1683412016368,
 'extendedChange': 0.14,
 'extendedChangePercent': 0.00103,
 'extendedPrice': 143.08,
 'extendedPriceTime': 1676216866020,
 'high': 151.32,
 'highSource': 'pereeda1tmdiciln y5 e u',
 'highTime': 1714597741035,
 'iexAskPrice': 0,
 'iexAskSize': 0,
 'iexBidPrice': 0,
 'iexBidSize': 0,
 'iexClose': 142.84,
 'iexCloseTime': 1689536952549,
 'iexLastUpdated': 1745891930933,
 'iexMarketPercent': 0.02120132085728544,
 'iexOpen': 149.7,
 'iexOpenTime': 1674832672183,
 'iexRealtimePrice': 145.1,
 'iexRealtimeSize': 419,
 'iexVolume': 1802802,
 'lastTradeTime': 1739724489147,
 'latestPrice': 146.56,
 'latestSource': 'Close',
 'latestTime': 'November 29, 2022',
 'latestUpdate': 1744975544192,
 'latestVolume': 869

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [7]:
price = data['latestPrice']
market_cap = data['marketCap']
market_cap / 1000000000000

2.2507101023

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [8]:
my_columns = ['Ticker','Stock Price','Market Capitilazation','Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns=my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitilazation,Number of Shares to Buy


In [9]:
row = pd.DataFrame([symbol,price,market_cap,'N/A'],index=my_columns).T
row

,Ticker,Stock Price,Market Capitilazation,Number of Shares to Buy
0,AAPL,146.56,2250710102300,N/A


In [10]:
final_dataframe = pd.concat([final_dataframe, row], ignore_index=True,
                            axis=0)
final_dataframe

,Ticker,Stock Price,Market Capitilazation,Number of Shares to Buy
0,AAPL,146.56,2250710102300,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [ ]:
final_dataframe = pd.DataFrame(columns=my_columns)
for stock in stocks['Ticker']:
    try:
        print(stock)
        api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
        data = requests.get(api_url).json()
        symbol= data['symbol']
        price = data['latestPrice']
        market_cap = data['marketCap']
        market_cap /= 1000000000000
        row = pd.DataFrame([symbol,price,market_cap,'N/A'],index=my_columns).T
        final_dataframe = pd.concat([final_dataframe, row], ignore_index=True,
                                axis=0)
    except:
        print('err has occured')
    

A
AAL
AAP
AAPL
ABBV
ABC
err has occured
ABMD
ABT
ACN
ADBE
ADI
ADM
ADP
ADSK
AEE
AEP
AES
AFL
AIG
AIV
AIZ
AJG
err has occured
AKAM
ALB
ALGN
ALK
ALL
err has occured
ALLE
ALXN
AMAT
AMCR
AMD
AME
AMGN
AMP
AMT
AMZN


In [11]:
final_dataframe

,Ticker,Stock Price,Market Capitilazation,Number of Shares to Buy
0,AAPL,146.56,2250710102300,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [12]:
def chunks(lst,n):
    for i in range(0,len(lst),n):
        yield lst[i:i+n]

In [13]:
symbol_groups = list(chunks(stocks['Ticker'],100))
symbol_strings = []

for i in range(0,len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

final_dataframe = pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
#     print(batch_api_call_url)
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        try:
            row = pd.DataFrame([symbol,data[symbol]['quote']['latestPrice']
                                ,data[symbol]['quote']['marketCap'],'N/A'],
                               index=my_columns).T
            final_dataframe = pd.concat([final_dataframe, row], ignore_index=True,
                                    axis=0)
        except: 
            print('one err')


one err
one err
one err
one err


In [14]:
final_dataframe

,Ticker,Stock Price,Market Capitilazation,Number of Shares to Buy
0,A,156.41,45034211807,N/A
1,AAL,14.71,9489652425,N/A
2,AAP,149.46,9194842007,N/A
3,AAPL,144.95,2325599747198,N/A
4,ABBV,161.3,289468837445,N/A
...,...,...,...,...
496,YUM,130.25,36670814809,N/A
497,ZBH,119,25331306371,N/A
498,ZBRA,275.47,13999636813,N/A
499,ZION,51.64,8011202856,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [15]:
while(True):
    portfolio_size = input("Enter the value of your portfolio:")    
    try:
        val = float(portfolio_size)
        print(val)
        break
    except ValueError:
        print('Enter only numbers')
        continue

Enter the value of your portfolio:100000
100000.0


In [16]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])-1):
    final_dataframe.loc[i, 
'Number of Shares to Buy'] = math.floor(position_size/final_dataframe['Stock Price'][i])
final_dataframe

,Ticker,Stock Price,Market Capitilazation,Number of Shares to Buy
0,A,156.41,45034211807,1
1,AAL,14.71,9489652425,13
2,AAP,149.46,9194842007,1
3,AAPL,144.95,2325599747198,1
4,ABBV,161.3,289468837445,1
...,...,...,...,...
496,YUM,130.25,36670814809,1
497,ZBH,119,25331306371,1
498,ZBRA,275.47,13999636813,0
499,ZION,51.64,8011202856,3


In [17]:
position_size

199.6007984031936

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [18]:
writer = pd.ExcelWriter('recommended trades.xlsx',engine='xlsxwriter')
final_dataframe.to_excel(writer,'Recommended Trades',index=False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [19]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [20]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

## Saving Our Excel Output

Saving our Excel file is very easy:

In [21]:
writer.save()

/tmp/ipykernel_5807/934276808.py:1: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
